In [66]:
import numpy as np
import scipy as sp
import sklearn
import nltk
import tensorflow as tf
import matplotlib
import gzip
import random
from collections import defaultdict

regularization_rate = 0.01
Reduction_Size = 20
batch_size = 200
max_iter = 50000
learning_rate = 0.0001

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
    

In [67]:
data = []
for l in readGz('train.json.gz'):
    data.append(l)

In [68]:
userList = []
businessList = []
userDict = defaultdict(int)
businessDict = defaultdict(int)
visitedDict = defaultdict(int)
for d in data:
    u = d['userID']
    b = d['businessID']
    if u not in userList:
        userList.append(u)
    if b not in businessList:
        businessList.append(b)
    visitedDict[(u,b)]+=1

for i in range(len(userList)):
    userDict[userList[i]] = i
for i in range(len(businessList)):
    businessDict[businessList[i]] = i

In [69]:
random.shuffle(data)
train_data = data[0:160000]
validation_data = data[160000:200000]
negative_pair = []
randomCnt = 0
while(randomCnt) < 40000:
    ruIndex = random.randint(0, len(userList)-1)
    rbIndex = random.randint(0, len(businessList)-1)
    ru = userList[ruIndex]
    rb = businessList[rbIndex]
    if visitedDict[(ru, rb)]==0:
        randomCnt +=1
        negative_pair.append((ru,rb))
#Finished creating dataset

ratings = np.array([d['rating'] for d in train_data])
avgRating = np.mean(ratings)
print avgRating


4.18365


In [72]:
betau = tf.get_variable('betau',dtype = tf.float32, shape = (len(userList)), initializer = tf.random_normal_initializer(stddev = 0.1))
tf.add_to_collection("Regularize", regularizer(betau))
   
betai = tf.get_variable('betai',dtype = tf.float32, shape = (len(businessList)), initializer = tf.random_normal_initializer(stddev = 0.1))
tf.add_to_collection("Regularize", regularizer(betai))


gammai = tf.get_variable('gammai', dtype = tf.float32, shape = (len(businessList), Reduction_Size), initializer = tf.random_normal_initializer(0.1))
gammau = tf.get_variable('gammau', dtype = tf.float32, shape = (len(userList), Reduction_Size), initializer = tf.random_normal_initializer(0.1))
tf.add_to_collection("Regularize", regularizer(gammai))
tf.add_to_collection("Regularize", regularizer(gammau))

In [74]:
#create calculation graph for tensorflow
#X is of a batch size
def calc(X, regularizer):    
    return [avgRating+betau[u]+betai[i]+tf.matmul(gammai[i].gammau[u].T) for x in X]

In [75]:
#put the training process here
train_feature = [[userDict[d['userID']], businessDict[d['businessID']]] for d in train_data]
train_feature = np.matrix(train_feature)
print train_feature[:10]
train_rating = np.array([d['rating'] for d in train_data])

X = tf.placeholder(tf.int32, shape = (batch_size, 2), name = 'X')
y = tf.placeholder(tf.float32, shape = (batch_size), name = 'y')

regularizer = tf.contrib.layers.l2_regularizer(scale = regularization_rate)

R_predict = calc(X, regularizer)
loss = tf.reduce_mean(tf.square(R_predict-y))
loss += tf.add_n(tf.get_collection("Regularize"))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)


for i in range(max_iter):
    with tf.Session() as sess:
        sess.initialize_all_variables.run()
        X_batch = random.sample(train_feature, batch_size)

[[   68  4207]
 [ 2379  6285]
 [  268 16398]
 [18181   154]
 [11415  9193]
 [ 7227 17711]
 [15235   181]
 [ 9273  4781]
 [13757  1961]
 [18510 19866]]


TypeError: 'Tensor' object is not iterable.

In [6]:
predictions = open("predictions_Visit.txt", 'w')
for l in open("pairs_Visit.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  if i in return1:
    predictions.write(u + '-' + i + ",1\n")
  else:
    predictions.write(u + '-' + i + ",0\n")

predictions.close()

NameError: name 'return1' is not defined